In [1]:
import pandas as pd
from utils.watchlist import generate_watchlist

In [2]:
df = pd.read_parquet(
    'https://objectstorage.ap-hyderabad-1.oraclecloud.com/n/axbaetdfzydd/b/terminal-files/o/symbols-full-v2.parquet')

In [3]:
rt = generate_watchlist(df,'industry_2')

/Users/farmaan/Public/farmaan/code/terminal/terminal-data/utils/watchlist.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = filtered_df.groupby(group_by).apply(format_group)


In [4]:
rt

['###Abrasives And Grinding Wheels,NSE:GRINDWELL+NSE:CARBORUNIV+NSE:WENDT/3,NSE:GRINDWELL,NSE:CARBORUNIV,NSE:WENDT,###Advertisement,NSE:AFFLE+NSE:SIGNPOST/2,NSE:AFFLE,NSE:SIGNPOST,###Aerospace & Defence- Equipments,NSE:HAL+NSE:BEL+NSE:BEML+NSE:ZENTEC+NSE:DATAPATTNS+NSE:ASTRAMICRO+NSE:MIDHANI+NSE:PARAS+NSE:AXISCADES+NSE:APOLLO/17,NSE:HAL,NSE:BEL,NSE:BEML,NSE:ZENTEC,NSE:DATAPATTNS,NSE:ASTRAMICRO,NSE:MIDHANI,NSE:PARAS,NSE:AXISCADES,NSE:APOLLO,NSE:DYNAMATECH,NSE:MTARTECH,NSE:AVANTEL,NSE:DCXINDIA,NSE:IDEAFORGE,NSE:NIBE,NSE:ROSSTECH,###Agri- Processing,NSE:CCL,###Agricultural Products,NSE:AVTNPL,###Agro Chemicals/Pesticides,NSE:PIIND+NSE:UPL+NSE:BAYERCROP+NSE:SUMICHEM+NSE:JUBLINGREA+NSE:EPIGRAL+NSE:SHARDACROP+NSE:DHANUKA+NSE:RALLIS+NSE:NACLIND/22,NSE:PIIND,NSE:UPL,NSE:BAYERCROP,NSE:SUMICHEM,NSE:JUBLINGREA,NSE:EPIGRAL,NSE:SHARDACROP,NSE:DHANUKA,NSE:RALLIS,NSE:NACLIND,NSE:BHARATRAS,NSE:BHAGCHEM,NSE:INSECTICID,NSE:MOL,NSE:IPL,NSE:ASTEC,NSE:EXCELINDUS,NSE:HERANBA,NSE:PUNJABCHEM,NSE:DHARMAJ,NSE:B

In [47]:
def trading_view_watchlist_generate(df: pd.DataFrame, group_by: str):
    filtered_df = df[df['mcap'] > 9e9].copy()

    # Group by industry_2, get top 10 by mcap, and create the formatted string with count
    def format_group(group):
        # Skip groups with only 1 item
        if len(group) <= 1:
            return group.index.astype(str)

        # Get top 10 by market cap
        top_10 = group.nlargest(10, 'mcap')
        # Join the tickers
        tickers = "+".join(top_10.index.astype(str))
        # Get total count in the filtered group
        total_count = len(group)
        return [f"{tickers}/{total_count}", *group.index.astype(str)]

    grouped_df = filtered_df.groupby('sub_industry').apply(format_group)
    # Remove None values (groups with only 1 item)
    grouped_df = grouped_df.dropna()
    final_string = ','.join([f"###{index},{",".join(value)}" for index, value in grouped_df.items()])
    return final_string


In [48]:
f = trading_view_watchlist_generate(df, 'sub_industry')

/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_6121/1411193519.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = filtered_df.groupby('sub_industry').apply(format_group)
/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_6121/1411193519.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = filtered_df.groupby('sub_industry').apply(format_group)


In [49]:
f

'###2/3 Wheelers,NSE:EICHERMOT+NSE:TVSMOTOR+NSE:HEROMOTOCO+NSE:OLAELEC/4,NSE:EICHERMOT,NSE:TVSMOTOR,NSE:HEROMOTOCO,NSE:OLAELEC,###Abrasives & Bearings,NSE:TIMKEN+NSE:SKFINDIA+NSE:GRINDWELL+NSE:CARBORUNIV+NSE:WENDT/5,NSE:TIMKEN,NSE:SKFINDIA,NSE:GRINDWELL,NSE:CARBORUNIV,NSE:WENDT,###Advertising & Media Agencies,NSE:SIGNPOST,###Aerospace & Defense,NSE:HAL+NSE:BEL+NSE:BDL+NSE:GRSE+NSE:ZENTEC+NSE:DATAPATTNS+NSE:ASTRAMICRO+NSE:MIDHANI+NSE:PARAS+NSE:UNIMECH/19,NSE:HAL,NSE:BEL,NSE:BDL,NSE:GRSE,NSE:ZENTEC,NSE:DATAPATTNS,NSE:ASTRAMICRO,NSE:MIDHANI,NSE:PARAS,NSE:UNIMECH,NSE:AXISCADES,NSE:APOLLO,NSE:MTARTECH,NSE:AVANTEL,NSE:CYIENTDLM,NSE:DCXINDIA,NSE:IDEAFORGE,NSE:NIBE,NSE:ROSSTECH,###Airline,NSE:INDIGO,###Airport & Airport services,NSE:GMRAIRPORT,###Aluminium,NSE:HINDALCO+NSE:NATIONALUM/2,NSE:HINDALCO,NSE:NATIONALUM,###Aluminium Copper & Zinc Products,NSE:PRECWIRE,###Amusement Parks/ Other Recreation,NSE:WONDERLA+NSE:IMAGICAA+NSE:DELTACORP/3,NSE:WONDERLA,NSE:IMAGICAA,NSE:DELTACORP,###Animal Feed,

In [42]:
filtered_df = df[df['mcap'] > 3e9].copy()


# Group by industry_2, get top 10 by mcap, and create the formatted string with count
def format_group(group):
    # Skip groups with only 1 item
    if len(group) <= 1:
        return None

    # Get top 10 by market cap
    top_10 = group.nlargest(10, 'mcap')
    # Join the tickers
    tickers = "+".join(top_10.index.astype(str))
    # Get total count in the filtered group
    total_count = len(group)
    return ",".join([f"{tickers}/{total_count}", *group.index.astype(str)])


grouped_df = filtered_df.groupby('sub_industry').apply(format_group)

# Remove None values (groups with only 1 item)
grouped_df = grouped_df.dropna()

# Convert to final string format
final_string = ','.join([f"###{index},{value}" for index, value in grouped_df.items()])


/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_6121/1955051059.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = filtered_df.groupby('sub_industry').apply(format_group)
/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_6121/1955051059.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = filtered_df.groupby('sub_industry').apply(format_group)


In [43]:
final_string

'###2/3 Wheelers,NSE:EICHERMOT+NSE:TVSMOTOR+NSE:HEROMOTOCO+NSE:OLAELEC/4,###Abrasives & Bearings,NSE:TIMKEN+NSE:SKFINDIA+NSE:GRINDWELL+NSE:CARBORUNIV+NSE:WENDT/5,###Advertising & Media Agencies,NSE:SIGNPOST+NSE:RKSWAMY+NSE:VERTOZ/3,###Aerospace & Defense,NSE:HAL+NSE:BEL+NSE:BDL+NSE:GRSE+NSE:ZENTEC+NSE:DATAPATTNS+NSE:ASTRAMICRO+NSE:MIDHANI+NSE:PARAS+NSE:UNIMECH/19,###Airline,NSE:INDIGO+NSE:GLOBALVECT/2,###Airport & Airport services,NSE:GMRAIRPORT+NSE:DREAMFOLKS/2,###Aluminium,NSE:HINDALCO+NSE:NATIONALUM+NSE:MMP+NSE:MAANALU/4,###Amusement Parks/ Other Recreation,NSE:WONDERLA+NSE:IMAGICAA+NSE:DELTACORP/3,###Animal Feed,NSE:GODREJAGRO+NSE:AVANTIFEED+NSE:MUKKA/3,###Asset Management Company,NSE:HDFCAMC+NSE:ABSLAMC+NSE:UTIAMC/3,###Auto Components & Equipments,NSE:BOSCHLTD+NSE:SCHAEFFLER+NSE:UNOMINDA+NSE:BHARATFORG+NSE:TIINDIA+NSE:ENDURANCE+NSE:EXIDEIND+NSE:MSUMI+NSE:SONACOMS+NSE:ZFCVINDIA/76,###Auto Dealer,NSE:LANDMARK+NSE:PVSL/2,###Breweries & Distilleries,NSE:UNITDSPR+NSE:UBL+NSE:RADICO+NSE